In [2]:
import pandas as pd
import geopandas as gpd
import requests
import zipfile
from shapely.geometry import Point, LineString
import numpy as np
import osmnx as ox
import multiprocessing
import pickle

# Get data

In [2]:
# data_url = "https://dev.hsl.fi/citybikes/od-trips-2024/od-trips-2024.zip"
# zip_path = "../data/od-trips-2024.zip"

# r = requests.get(data_url)
# with open(zip_path, "wb") as f:
#     f.write(r.content)

In [3]:
# dfs = []

# with zipfile.ZipFile(zip_path, 'r') as zip_ref:
#     for file_name in zip_ref.namelist()[1:]:
#         with zip_ref.open(file_name) as file:
#             df = pd.read_csv(file)
#             dfs.append(df)

# trip_df = pd.concat(dfs)

In [4]:
# trip_df.to_csv("../data/raw/trips.csv")

In [ ]:
trips_df = pd.read_csv("../data/processed/trips.csv")
stations_df = pd.read_csv("../data/raw/Helsingin_ja_Espoon_kaupunkipyöräasemat_avoin_7704606743268189464.csv")

In [3]:
pd.read_csv("../data/processed/station_pair_counts.csv")

,Departure station id,Return station id,count,x_dep,y_dep,x_ret,y_ret
0,1,1,1218,24.950211,60.155370,24.950211,60.155370
1,1,2,156,24.950211,60.155370,24.956510,60.160989
2,1,3,141,24.950211,60.155370,24.945018,60.158177
3,1,4,240,24.950211,60.155370,24.941776,60.160986
4,1,5,246,24.950211,60.155370,24.936285,60.157948
...,...,...,...,...,...,...,...
84005,902,753,1,24.835356,60.209017,24.813954,60.234401
84006,902,757,4,24.835356,60.209017,24.831397,60.235089
84007,902,761,32,24.835356,60.209017,24.839390,60.220467
84008,902,763,22,24.835356,60.209017,24.833536,60.220551


In [37]:
stations_df["geometry"] = [Point(xy) for xy in zip(stations_df.x, stations_df.y)]
stations_df

,FID,ID,Nimi,Namn,Name,Osoite,Adress,Kaupunki,Stad,Operaattor,Kapasiteet,x,y,x2,y2,geometry
0,1,501,Hanasaari,Hanaholmen,Hanasaari,Hanasaarenranta 1,Hanaholmsstranden 1,Espoo,Esbo,CityBike Finland,10,24.840319,60.165820,2.549113e+07,6.672558e+06,POINT (24.840319 60.16582)
1,2,503,Keilalahti,Kägelviken,Keilalahti,Keilalahdentie 2,Kägelviksvägen 2,Espoo,Esbo,CityBike Finland,28,24.827467,60.171524,2.549042e+07,6.673195e+06,POINT (24.827467 60.171524)
2,3,505,Westendinasema,Westendstationen,Westendinasema,Westendintie 1,Westendvägen 1,Espoo,Esbo,CityBike Finland,16,24.805758,60.168266,2.548922e+07,6.672836e+06,POINT (24.805758 60.168266)
3,4,507,Golfpolku,Golfstigen,Golfpolku,Golfpolku 3,Golfstigen 3,Espoo,Esbo,CityBike Finland,16,24.796136,60.168143,2.548868e+07,6.672824e+06,POINT (24.796136 60.168143)
4,5,509,Revontulentie,Norrskensvägen,Revontulentie,Revontulentie 10,Norrskensvägen 10,Espoo,Esbo,CityBike Finland,30,24.802938,60.171551,2.548906e+07,6.673202e+06,POINT (24.802938 60.171551)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
452,453,401,Koivusaari (M),Björkholmens metrostation,Koivusaari metro station,Sotkatie 11,Dykandsvägen 11,,,,16,24.855164,60.163386,2.549196e+07,6.672285e+06,POINT (24.8551642686834 60.1633863116225)
453,454,402,Bermudankuja,Bermudagränden,Välimerenkatu,Atlantinkatu 10,Atlantgatan 10,,,,16,24.910142,60.154904,2.549501e+07,6.671335e+06,POINT (24.9101421129463 60.1549042600429)
454,455,403,Verkkosaari,Nätholmen,Verkkosaari,Tukkutorinkuja 3,Partitorgsgränden,,,,20,24.977548,60.189556,2.549875e+07,6.675192e+06,POINT (24.9775480765957 60.1895561753156)
455,456,404,Sompasaari,Sumparn,Sompasaari,Sompasaarenlaituri 2,Sumparkajen 2,,,,14,24.976076,60.182931,2.549867e+07,6.674454e+06,POINT (24.9760758238593 60.1829308185667)


In [38]:
grouped_counts_df= trips_df.copy()
grouped_counts_df["Departure"] = pd.to_datetime(grouped_counts_df["Departure"], format="mixed")
grouped_counts_df["time"] = grouped_counts_df["Departure"].dt.round(freq="60min").dt.time
grouped_counts_df

,Departure,Return,Departure station id,Departure station name,Return station id,Return station name,Covered distance (m),Duration (sec.),time
0,2024-04-30 23:59:23,2024-05-01T00:31:48,18,Porthania,103,Korppaantie,6115.0,1941,00:00:00
1,2024-04-30 23:59:08,2024-05-01T00:01:25,259,Petter Wetterin tie,255,Laivalahden puistotie,614.0,133,00:00:00
2,2024-04-30 23:58:52,2024-05-01T00:03:26,36,Apollonkatu,77,Nordenskiöldinaukio,1108.0,270,00:00:00
3,2024-04-30 23:58:51,2024-05-01T00:15:50,294,Postipuisto,237,Aulangontie,4135.0,1002,00:00:00
4,2024-04-30 23:58:50,2024-05-01T00:22:30,44,Sörnäinen (M),116,Linnanmäki,1480.0,1415,00:00:00
...,...,...,...,...,...,...,...,...,...
2585663,2024-10-01 00:04:58,2024-10-01T00:12:40,135,Velodrominrinne,204,A.I. Virtasen aukio,1331.0,458,00:00:00
2585664,2024-10-01 00:04:23,2024-10-01T00:09:38,263,Herttoniemen kirkko,264,Eränkävijäntori,826.0,314,00:00:00
2585665,2024-10-01 00:02:52,2024-10-01T00:07:19,129,Pernajantie,44,Sörnäinen (M),937.0,261,00:00:00
2585666,2024-10-01 00:00:36,2024-10-01T00:17:21,401,Koivusaari (M),541,"Aalto-yliopisto (M), Korkeakouluaukio",3761.0,1000,00:00:00


In [39]:
grouped_counts_df = grouped_counts_df.groupby(["Departure station id", "Return station id", "time"])["Return"].count().reset_index().rename(columns={"Return": "count"})
grouped_counts_df["ids"] = grouped_counts_df.apply(lambda row: tuple(sorted((int(row["Departure station id"]), int(row["Return station id"])))), axis=1)
grouped_counts_df

,Departure station id,Return station id,time,count,ids
0,1,1,00:00:00,25,"(1, 1)"
1,1,1,01:00:00,7,"(1, 1)"
2,1,1,02:00:00,10,"(1, 1)"
3,1,1,03:00:00,3,"(1, 1)"
4,1,1,04:00:00,2,"(1, 1)"
...,...,...,...,...,...
528441,999,312,07:00:00,1,"(312, 999)"
528442,999,364,23:00:00,1,"(364, 999)"
528443,999,573,20:00:00,1,"(573, 999)"
528444,999,627,18:00:00,1,"(627, 999)"


In [40]:
grouped_counts_df.to_csv("../data/processed/grouped_counts.csv", index=False)

In [7]:
trips_df = trips_df[trips_df["Return station id"] < 997]
trips_df = trips_df[trips_df["Departure station id"] < 997]

In [ ]:
station_pair_counts = trips_df.groupby(["Departure station id", "Return station id"])["Departure"].count().reset_index().rename(columns={"Departure":"count"})
station_pair_counts = station_pair_counts.merge(stations_df[["ID", "x", "y"]], left_on="Departure station id", right_on="ID")
station_pair_counts = station_pair_counts.merge(stations_df[["ID", "x", "y"]], left_on="Return station id", right_on="ID", suffixes=("_dep", "_ret"))
station_pair_counts = station_pair_counts.drop(columns=["ID_dep", "ID_ret"])
# # station_pair_counts = station_pair_counts.drop(columns=["ID_dep", "ID_ret"])
# station_pair_counts["lines"] = [LineString(xy) for xy in zip(station_pair_counts.geometry_dep, station_pair_counts.geometry_ret)]

# station_pair_counts["ids"] = station_pair_counts.apply(lambda row: tuple(sorted((int(row["Departure station id"]), int(row["Return station id"])))), axis=1)

station_pair_counts

,Departure station id,Return station id,count,x_dep,y_dep,x_ret,y_ret,ids
0,1,1,1218,24.950211,60.155370,24.950211,60.155370,"(1, 1)"
1,1,2,156,24.950211,60.155370,24.956510,60.160989,"(1, 2)"
2,1,3,141,24.950211,60.155370,24.945018,60.158177,"(1, 3)"
3,1,4,240,24.950211,60.155370,24.941776,60.160986,"(1, 4)"
4,1,5,246,24.950211,60.155370,24.936285,60.157948,"(1, 5)"
...,...,...,...,...,...,...,...,...
84005,902,753,1,24.835356,60.209017,24.813954,60.234401,"(753, 902)"
84006,902,757,4,24.835356,60.209017,24.831397,60.235089,"(757, 902)"
84007,902,761,32,24.835356,60.209017,24.839390,60.220467,"(761, 902)"
84008,902,763,22,24.835356,60.209017,24.833536,60.220551,"(763, 902)"


In [32]:
df = station_pair_counts[:5]
df["test"] = [1,2,3,4,5]
df.set_index("ids")["test"].to_dict()

C:\Users\pyryp\AppData\Local\Temp\ipykernel_12764\463179979.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["test"] = [1,2,3,4,5]


{(1, 1): 1, (1, 2): 2, (1, 3): 3, (1, 4): 4, (1, 5): 5}

In [ ]:
station_pair_counts.drop_duplicates("ids")

In [9]:
df = station_pair_counts[station_pair_counts["Departure station id"]==1]
df = df[df["count"]>10]
df["count"] = np.log2(df["count"])
gpd.GeoDataFrame(df, geometry="lines", crs="EPSG:4326").explore(column=df["count"], cmap="Blues")

# Paths

In [10]:
dep_points_x_list = station_pair_counts["geometry_dep"].apply(lambda point: point.x).tolist()
dep_points_y_list = station_pair_counts["geometry_dep"].apply(lambda point: point.y).tolist()

ret_points_x_list = station_pair_counts["geometry_ret"].apply(lambda point: point.x).tolist()
ret_points_y_list = station_pair_counts["geometry_ret"].apply(lambda point: point.y).tolist()

left = min(dep_points_x_list)
bottom = min(dep_points_y_list)
right = max(dep_points_x_list)
top = max(dep_points_y_list)

In [11]:
# G = ox.graph_from_bbox((left, bottom, right, top), network_type='bike')
# with open("../data/graph.pkl", "wb") as f:
#     pickle.dump(G, f)

with open("../data/graph.pkl", "rb") as f:
    G = pickle.load(f)

In [12]:
orig = ox.nearest_nodes(G, dep_points_x_list, dep_points_y_list)
dest = ox.nearest_nodes(G, ret_points_x_list, ret_points_y_list)

In [13]:
args = [(i, G, orig, dest) for i in range(0, len(orig), 6000)]
# args = [(i, G, orig, dest) for i in range(0, 300, 100)]

In [14]:
import functions
with multiprocessing.Pool(14) as pool:
    results = pool.starmap(functions.router, args)

In [55]:
routes = [value for d in results for value in d.values()]

In [58]:
routes = [x for xs in routes for x in xs]

In [59]:
route_coords = [[(G.nodes[node]["x"], G.nodes[node]["y"]) for node in route] for route in routes]
linestrings = [LineString(coords) if len(coords) > 1 else Point(coords) for coords in route_coords]

In [75]:
i = 60000
gpd.GeoDataFrame(geometry=linestrings[i:i+100], crs="EPSG:4326").explore(color="red", tiles="Cartodb Positron")

In [85]:
journey_routes_df = station_pair_counts.set_geometry(linestrings)
journey_routes_df = journey_routes_df.set_crs("EPSG:4326")
journey_routes_df

,Departure station id,Return station id,count,ID_dep,geometry_dep,ID_ret,geometry_ret,lines,geometry
0,1,1,1218,1,POINT (24.9502114714031 60.155369615074),1,POINT (24.9502114714031 60.155369615074),"LINESTRING (24.9502114714031 60.155369615074, ...",POINT (24.95027 60.15524)
1,1,2,156,1,POINT (24.9502114714031 60.155369615074),2,POINT (24.9565097715858 60.1609890692806),"LINESTRING (24.9502114714031 60.155369615074, ...","LINESTRING (24.95027 60.15524, 24.95079 60.155..."
2,1,3,141,1,POINT (24.9502114714031 60.155369615074),3,POINT (24.9450181631667 60.1581769029949),"LINESTRING (24.9502114714031 60.155369615074, ...","LINESTRING (24.95027 60.15524, 24.95079 60.155..."
3,1,4,240,1,POINT (24.9502114714031 60.155369615074),4,POINT (24.9417758006418 60.1609858921764),"LINESTRING (24.9502114714031 60.155369615074, ...","LINESTRING (24.95027 60.15524, 24.95079 60.155..."
4,1,5,246,1,POINT (24.9502114714031 60.155369615074),5,POINT (24.9362853002102 60.157948291819),"LINESTRING (24.9502114714031 60.155369615074, ...","LINESTRING (24.95027 60.15524, 24.95079 60.155..."
...,...,...,...,...,...,...,...,...,...
84005,902,753,1,902,POINT (24.835356 60.209017),753,POINT (24.813954 60.234401),"LINESTRING (24.835356 60.209017, 24.813954 60....","LINESTRING (24.83519 60.20903, 24.8349 60.2091..."
84006,902,757,4,902,POINT (24.835356 60.209017),757,POINT (24.831397 60.235089),"LINESTRING (24.835356 60.209017, 24.831397 60....","LINESTRING (24.83519 60.20903, 24.8349 60.2091..."
84007,902,761,32,902,POINT (24.835356 60.209017),761,POINT (24.83939 60.220467),"LINESTRING (24.835356 60.209017, 24.83939 60.2...","LINESTRING (24.83519 60.20903, 24.8349 60.2091..."
84008,902,763,22,902,POINT (24.835356 60.209017),763,POINT (24.833536 60.220551),"LINESTRING (24.835356 60.209017, 24.833536 60....","LINESTRING (24.83519 60.20903, 24.8349 60.2091..."


In [86]:
journey_routes_df.to_file("../data/processed/journey_routes.geojson")

In [87]:
journey_routes_df.to_file("../data/processed/journey_routes.gpkg")

In [65]:
gdf = station_pair_counts[:300]
gdf = gdf.set_geometry(linestrings)
gdf = gdf.set_crs("EPSG:4326")

gdf = gdf[gdf["Departure station id"]==1]
gdf["count"] = np.log2(gdf["count"])
gdf = gdf.sort_values(by="count", ascending=False)

gdf.explore(column="count", tiles="Cartodb Positron", cmap="Oranges")

In [67]:
gdf[["count", "geometry"]]

,count,geometry
0,10.250298,POINT (24.95027 60.15524)
11,9.588715,"LINESTRING (24.95027 60.15524, 24.95079 60.155..."
29,9.187352,"LINESTRING (24.95027 60.15524, 24.95079 60.155..."
5,9.182394,"LINESTRING (24.95027 60.15524, 24.95079 60.155..."
10,8.962896,"LINESTRING (24.95027 60.15524, 24.95079 60.155..."
...,...,...
244,0.000000,"LINESTRING (24.95027 60.15524, 24.95079 60.155..."
246,0.000000,"LINESTRING (24.95027 60.15524, 24.95079 60.155..."
245,0.000000,"LINESTRING (24.95027 60.15524, 24.95079 60.155..."
249,0.000000,"LINESTRING (24.95027 60.15524, 24.95079 60.155..."


In [47]:
gpd.read_file("../data/processed/journey_routes.gpkg").explore()